# H&M Personalized Fashion Recommendations
- 日本語コメント付きのベースラインモデルです
- 以下のnotebookをリファクタしたものになります
    - 元notebook
        - H&M Pure Pytorch Baseline
        - https://www.kaggle.com/code/aerdem4/h-m-pure-pytorch-baseline
    - リファクタ内容
        - DEBUGモード追加
        - コメント付与
        - 定数のみ切り出し

In [1]:
import numpy as np
import pandas as pd

In [2]:
DEBUG = True

# ファイルパス
# baselineではTRANSACTIONのみ学習に使用している
TRANSACTION_PATH = "../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv"
ARTICLES_PATH = "../input/h-and-m-personalized-fashion-recommendations/articles.csv"
CUSTOMERS_PATH = "../input/h-and-m-personalized-fashion-recommendations/customers.csv"
SAMPLE_SUB_PATH = '../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'
IMAGES_DIR = "../input/h-and-m-personalized-fashion-recommendations/images/"

# 前処理
ACTIVE_DATE_TH = "2019-09-01" # この日付より最近購入された商品のみ学習/推論対象とする
WEEK_HIST_MAX = 5 # 過去何日分の購買履歴を特徴量とするか
VAL_WEEKS = [0] # どの週のデータをvalidationデータとするか
TRAIN_WEEKS = [1, 2, 3, 4] # どの週のデータをtrainデータとするか

# データセット
SEQ_LEN = 16 # 何個分の購買履歴まで使うか
BS = 256 # バッチサイズ
NW = 8 # ワーカーズ

# 学習
DIM_ARTICLE = 512 # article_idを何次元の特徴量にembeddingするか
INIT_LR = 3e-4
BETAS = (0.9, 0.999)
EPS=1e-08
LR_DICT = { # n-epoch目の時、n < KEYならVALをlrとする
    1: 5e-5,
    6: 1e-3, 
    9: 1e-4,
    100: 1e-5
}
MODEL_NAME = "exp001"
SEED = 0
BASE_EPOCHS = 10 # ベースモデルのエポック数
FT_EPOCHS = 10 # fine tuningのエポック数
FT_WEEKS = 4 # fine tuningする際使用するweek数

In [3]:
# データ読み込み
df = pd.read_csv(TRANSACTION_PATH, dtype={"article_id": str})
print(df.shape)
df.head()

if DEBUG:
    print("DEBUGモードで処理を実行します")
    BASE_EPOCHS = 1
    FT_EPOCHS = 1
    SEQ_LEN = 5
    DIM_ARTICLE = 8
    df = df.sample(frac=0.005)
    print(df.shape)
    display(df.head())

(31788324, 5)
DEBUGモードで処理を実行します
(158942, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
22907892,2020-03-01,97d1bee94581b2d24b004b1f187114c42654a0e1fcae7d...,0842062003,0.033881,2
11463365,2019-06-06,a6582d4614cce36e1b4fe863e8afd452f1c08cccb18a1a...,0554598038,0.016254,1
14586750,2019-07-26,bf4a315fed4ab1b8dbe360c36253c0df4190bc7a5c2be3...,0634591006,0.025407,2
27360875,2020-06-17,3c81cb1e3bcae4970e327d8c1aa5569271a2940154ef62...,0882772001,0.033881,2
26604846,2020-05-30,ff59422d4070741a35dae45a168c61bfb882716ac52389...,0757926001,0.040661,2


# 前処理(1)

In [4]:
# datetime型に変換
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

Timestamp('2020-09-22 00:00:00')

In [5]:
# 直近で購入された商品一覧を抽出する
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= ACTIVE_DATE_TH].reset_index()
active_articles.shape

(22863, 3)

In [6]:
# 購買履歴のうち直近で購入された商品IDに対応する履歴のみを抽出
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

(108052, 5)

In [7]:
# "week"カラムを追加(weekが小さいほど最近)
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

13     2753
12     2608
42     2592
51     2167
23     2005
       ... 
102     263
93      256
90      245
94      226
97      195
Name: week, Length: 105, dtype: int64

In [8]:
# articleIdをラベルに変換
from sklearn.preprocessing import LabelEncoder
article_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])
le_article = LabelEncoder()
le_article.fit(article_ids)
df["article_id"] = le_article.transform(df["article_id"])

In [9]:
if DEBUG:
    print("データ(head)")
    display(df.head())
    print("データ(describe)")
    display(df.describe())
    print("データ(info)")
    display(df.info())

データ(head)


,t_dat,customer_id,article_id,price,sales_channel_id,week
0,2020-03-01,97d1bee94581b2d24b004b1f187114c42654a0e1fcae7d...,16917,0.033881,2,29
1,2019-06-06,a6582d4614cce36e1b4fe863e8afd452f1c08cccb18a1a...,1341,0.016254,1,67
2,2020-06-17,3c81cb1e3bcae4970e327d8c1aa5569271a2940154ef62...,20792,0.033881,2,13
3,2020-05-30,ff59422d4070741a35dae45a168c61bfb882716ac52389...,8671,0.040661,2,16
4,2020-03-10,9a6759a5e1d573f8906ce11a89f494b87d805c47c8e7a2...,14607,0.033000,2,28


データ(describe)


,article_id,price,sales_channel_id,week
count,108052.000000,108052.000000,108052.000000,108052.000000
mean,9110.163523,0.027530,1.706067,39.039971
std,6465.767451,0.018317,0.455564,25.800356
min,0.000000,0.000424,1.000000,0.000000
25%,3511.000000,0.016119,1.000000,16.000000
50%,7966.000000,0.025407,2.000000,37.000000
75%,14458.000000,0.033881,2.000000,58.000000
max,22862.000000,0.422034,2.000000,104.000000


データ(info)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108052 entries, 0 to 108051
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   t_dat             108052 non-null  datetime64[ns]
 1   customer_id       108052 non-null  object        
 2   article_id        108052 non-null  int64         
 3   price             108052 non-null  float64       
 4   sales_channel_id  108052 non-null  int64         
 5   week              108052 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 4.9+ MB


None

# 前処理(2)

In [10]:
def create_dataset(df, week, debug=False) -> pd.DataFrame:
    """
    データセットを作成
    各顧客が指定した週で購入したものと、過去n週で購入したもの
    
    【データの説明】
        customer_id: 顧客ID
        target: (その週に)何を購入したか←予測したいもの
        week: 週
        article_id: (過去n週で)何を購入したか
        week_history: (過去n週の)どの週で購入したか
        
    【処理の流れ】
    1.指定した週の過去n週間分のデータを抽出する
    2.顧客(customer_id)毎に買った商品リスト(article_idリスト)、購入したweekリストを作成
    3.購入したweekリストをweek_historyとする
    4.指定した週のレコードをtarget_dfとして抽出
    5.target_dfも顧客(customer_id)毎に買った商品リスト(article_idリスト)を作成
    6.買った商品リストのカラムをtargetに変換
    7.target_dfのweekカラムに指定した週の値を代入
    8.customer_idでleft join
    """
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    if debug:
        display(hist_df.head())
    
    target_df = df[df["week"] == week]
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    result_df = target_df.merge(hist_df, on="customer_id", how="left")
    if debug:
        display(result_df.head())
    return result_df

In [11]:
# データセット作成
val_df = pd.concat([create_dataset(df, w) for w in VAL_WEEKS]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in TRAIN_WEEKS]).reset_index(drop=True)
train_df.shape, val_df.shape

if DEBUG:
    print("validationデータ(head)")
    display(val_df.head())
    print("trainデータ(head + tail)")
    display(train_df.head())
    display(train_df.tail())

validationデータ(head)


,customer_id,target,week,article_id,week_history
0,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,[22789],0,NaN,NaN
1,001ddeb8fb74fec5693116da83b488e05ee9a9e179f3fd...,[20196],0,NaN,NaN
2,0026ebdd70715d8fa2befa14dfed317a1ffe5451aba839...,[11053],0,NaN,NaN
3,00940f3cc6e6ab97cfefa71b2bd479a5c0dbccf5b56f7a...,[21624],0,NaN,NaN
4,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,[3552],0,NaN,NaN


trainデータ(head + tail)


,customer_id,target,week,article_id,week_history
0,00b32956428a096cde500df8a2c97b3efe6acf4d183977...,[20087],1,NaN,NaN
1,00bd21cafc0274765423c9046d8c35c70a61471bf309b2...,[20921],1,NaN,NaN
2,00c142839dd8a8eb803280bcdb53f51f5f78f692c87f55...,[20663],1,NaN,NaN
3,0138bf975f6592f50c51ead422419e219c56b5defcf8c7...,[14337],1,NaN,NaN
4,0157365cc52095ead6207202d0cacb7ede8da16508bc18...,[19311],1,NaN,NaN


,customer_id,target,week,article_id,week_history
5198,ff6245022bb917b4fa09e809b23f4aa620db8dc5c214d2...,[20927],4,NaN,NaN
5199,ffa298b17476398c634bdaabab67562d03846736f55075...,[13112],4,NaN,NaN
5200,ffba9e1d82f8b430fbcc1241776fe58cb01525cf7d776f...,[22836],4,NaN,NaN
5201,ffc896b358b6690def53aef6acbed759b9980c341d8e6c...,[22232],4,NaN,NaN
5202,ffde1bab6b628c1f22f7658ffc16b65fa8713a3c3262f0...,[12965],4,NaN,NaN


# モデル学習

In [12]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    """
    pytorch Datasetを継承する際には以下の2つの関数をoverrideする
    ①.def __getitem__(self, index: int):
        indexを与えられたときに対応するデータを返す  
    ②.def __len__(self):
        データのサンプル数を返す
    """
    def __init__(self, df, seq_len, is_test=False):
        """
        df: 前処理したデータセット
        seq_len: 特徴量の次元数
        is_test: テストデータか否か
        """
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        """
        対象indexデータを加工して返す
        
        特徴量：
        article_hist: 過去n週で購買した商品(要素数=seq_len)
        week_hist: article_histの商品をいつ購入したかを正規化(0~1)した値(要素数=seq_len)
        目的変数: 
        target: この週どの商品を購入したか(要素数=len(article_ids))
        """
        
        # dfのindex番目の行を抽出
        row = self.df.iloc[index]
        
        if self.is_test:
            # testならtargetは長さ2のlist
            target = torch.zeros(2).float()
        else:
            # trainならtargetは長さが"対象とする商品数"のlist
            target = torch.zeros(len(article_ids)).float()
            
            # targetのうち顧客が購入したもの(article_idのindex)を1.0にする
            for t in row.target:
                target[t] = 1.0
            
        # 購入履歴は長さseq_lenのlist
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()
        
        # 何かしらの購入履歴があるか
        if isinstance(row.article_id, list):
            
            # 購入履歴の数がseq_lenより長いなら直近の履歴から抽出
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            # 購入履歴の数がseq_lenより短いならarticle_hist/week_histの前半indexに購入履歴を格納
            else:   
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2
                
        return article_hist, week_hist, target
    
if DEBUG:
    print("train_dfでHMDataSetを作成し中身を確認")
    print("[0]article_hist: 過去n週で購入した商品一覧")
    print("[1]week_hist: 各商品を過去n週のうちどの週で購入したか一覧")
    print("[2]target: 今週どの商品を購入したか0:購入しなかった、1:購入した(要素数: len(article_ids)")
    print("-"*60)
    print()
    print("データセットの中身")
    sample = HMDataset(train_df, 64)[0]
    display(sample)
    print()
    print("各特徴量のlen")
    print(len(sample[0]))
    print(len(sample[1]))
    print(len(sample[2]))

train_dfでHMDataSetを作成し中身を確認
[0]article_hist: 過去n週で購入した商品一覧
[1]week_hist: 各商品を過去n週のうちどの週で購入したか一覧
[2]target: 今週どの商品を購入したか0:購入しなかった、1:購入した(要素数: len(article_ids)
------------------------------------------------------------

データセットの中身


(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]))


各特徴量のlen
64
64
22864


In [13]:
def adjust_lr(optimizer, epoch):
    """
    エポックによって特徴量を変える
    """
    
    for epoch_th in LR_DICT.keys():
        if epoch < epoch_th:
            lr = LR_DICT[epoch_th]

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    """
    最適化関数を取得
    """
    optimizer = torch.optim.Adam(
        params=filter(lambda p: p.requires_grad, net.parameters()),
        lr=INIT_LR,
        betas=BETAS,
        eps=EPS
    )
    return optimizer

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class HMModel(nn.Module):
    """
    ネットワークを定義
    一方通行ではない複雑なモデル（ネットワーク）を構築するには、torch.nn.Moduleを継承したサブクラスを定義する
    
    基本的な書き方
    __init__()
    　- 使用するモジュール（レイヤー）のインスタンスを生成
     ※super().__init__()を忘れないように注意
    forward()
    　- レイヤーを所望の順番で適用していく
    """
    def __init__(self, article_shape):
        """
        article_shape = (商品の種類数, ベクトルの次元)
        """
        # おまじない
        super(HMModel, self).__init__()
        
        # レイヤー(Sequentialはレイヤーのひとまとまり?)
        # nn.Embedding: IDに対応する初期ベクトルを作成(IDをベクトルに埋め込む)
        # nn.Parameter: レイヤーのパラメータ
        self.article_emb = nn.Embedding(num_embeddings=article_shape[0], embedding_dim=article_shape[1])
        self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        self.top = nn.Sequential(
            nn.Conv1d(3, 32, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(32, 8, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(8, 1, kernel_size=1)
        )
        
    def forward(self, inputs):
        """
        inputs: 特徴量
        """
        article_hist, week_hist = inputs[0], inputs[1]
        
        # TODO: 前処理
        x = self.article_emb(article_hist)

        x = F.normalize(x, dim=2)
        x = x@F.normalize(self.article_emb.weight).T
        
        x, indices = x.max(axis=1)
        x = x.clamp(1e-3, 0.999)
        x = -torch.log(1/x - 1)
        
        max_week = week_hist.unsqueeze(2).repeat(1, 1, x.shape[-1]).gather(1, indices.unsqueeze(1).repeat(1, week_hist.shape[1], 1))
        max_week = max_week.mean(axis=1).unsqueeze(1)
        
        x = torch.cat([x.unsqueeze(1), max_week,
                       self.article_likelihood[None, None, :].repeat(x.shape[0], 1, 1)], axis=1)
        
        # ネットワークで計算
        x = self.top(x).squeeze(1)
        return x
    
    
model = HMModel((len(le_article.classes_), DIM_ARTICLE))
model = model.cuda()
if DEBUG:
    print("モデル構造を可視化")
    print(model)

モデル構造を可視化
HMModel(
  (article_emb): Embedding(22864, 8)
  (top): Sequential(
    (0): Conv1d(3, 32, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(32, 8, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(8, 1, kernel_size=(1,), stride=(1,))
  )
)


In [15]:
import sys

def calc_map(topk_preds, target_array, k=12):
    """
    MAPスコアを計算
    topk_perds: 予測結果
    target_array: 正解
    k: 予測件数(今回のコンペだと上位12件を予測)
    """
    metric = []
    tp, fp = 0, 0
    
    # 予測結果を1件ずつ見ていきtp/fpを計算
    for pred in topk_preds:
        if target_array[pred]:
            tp += 1
            metric.append(tp/(tp + fp))
        else:
            fp += 1
            
    return np.sum(metric) / min(k, target_array.sum())

def read_data(data):
    """
    TODO: おまじない的なやつ?
    """
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()


def validate(model, val_loader, k=12):
    """
    バリデーション
    """
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    maps = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            # inputs(入力特徴量),target(正解)を抽出
            inputs, target = read_data(data)
            
            # モデルで予測
            logits = model(inputs)
            
            # topk(スコア上位k件を抽出)
            _, indices = torch.topk(logits, k, dim=1)

            # GPU→CPUへ
            indices = indices.detach().cpu().numpy() # 予測結果
            target = target.detach().cpu().numpy() # 正解

            # スコア計算
            for i in range(indices.shape[0]):
                maps.append(calc_map(indices[i], target[i]))
        
    return np.mean(maps)

# DataSet: 学習に使うデータ(前処理などを定義している)
val_dataset = HMDataset(val_df, SEQ_LEN)

# DataLoader: DataSetをバッチで処理するためのもの
val_loader = DataLoader(
    val_dataset,
    batch_size=BS,
    shuffle=False,
    num_workers=NW,
    pin_memory=False,
    drop_last=False
)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### Train and validate

In [16]:
def dice_loss(y_pred, y_true):
    """
    dice_loss
    Dice損失は2つの要素の類似度の評価するために使われているDice係数(F値)を損失として用いたもの
    """
    y_pred = y_pred.sigmoid()
    intersect = (y_true*y_pred).sum(axis=1)
    
    return 1 - (intersect/(intersect + y_true.sum(axis=1) + y_pred.sum(axis=1))).mean()


def train(model, train_loader, val_loader, epochs):
    np.random.seed(SEED)
    
    # 最適化関数を定義
    optimizer = get_optimizer(model)
    scaler = torch.cuda.amp.GradScaler() # ←高速化のための何かっぽい

    # ロスを定義
    criterion = torch.nn.BCEWithLogitsLoss()
    
    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                logits = model(inputs)
                
                # ロスは2種類を組み合わせている(完全一致と類似度？)
                loss = criterion(logits, target) + dice_loss(logits, target)
            
            #loss.backward()
            scaler.scale(loss).backward()
            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()
            
            loss_list.append(loss.detach().cpu().item())
            
            avg_loss = np.round(100*np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
            
        val_map = validate(model, val_loader)

        log_text = f"Epoch {e+1}\nTrain Loss: {avg_loss}\nValidation MAP: {val_map}\n"
            
        print(log_text)
        
        #logfile = open(f"models/{MODEL_NAME}_{SEED}.txt", 'a')
        #logfile.write(log_text)
        #logfile.close()
    return model




train_dataset = HMDataset(train_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=BASE_EPOCHS)

100%|██████████| 5/5 [00:01<00:00,  4.05it/s]
Epoch 1
Train Loss: 167.1047
Validation MAP: 0.0008264462809917355



### Finetune with more recent data for submission (include validation set)
今回のコンペは購買履歴の次の週の結果を予測する  
→ 直近のデータの方が重要と考えられる  
→ 最後にバリデーションデータを使ってfine tuningし直す

In [17]:
train_dataset = HMDataset(train_df[train_df["week"] < FT_WEEKS].append(val_df), SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=FT_EPOCHS)

100%|██████████| 5/5 [00:01<00:00,  4.43it/s]
Epoch 1
Train Loss: 167.0082
Validation MAP: 0.0008264462809917355



In [18]:
test_df = pd.read_csv(SAMPLE_SUB_PATH).drop("prediction", axis=1)
print(test_df.shape)
test_df.head()

(1371980, 1)


,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [19]:
def create_test_dataset(test_df):
    """
    テストデータ(submitする際の予測データ)を作成する
    """
    week = -1
    test_df["week"] = week
    
    # 過去n週間で買ったものを列として追加していく
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    return test_df.merge(hist_df, on="customer_id", how="left")

test_df = create_test_dataset(test_df)
test_df.head()

,customer_id,week,article_id,week_history
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,-1,NaN,NaN
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-1,NaN,NaN
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,-1,NaN,NaN
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,-1,NaN,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,-1,NaN,NaN


In [20]:
# 8割くらいの人がnull
# →80%の人はヒントなしになってしまっているので別の特徴量が必要そう
test_df["article_id"].isnull().mean()

0.9953643639120103

# テストデータに対する推論 + submission.csvの作成

In [21]:
test_ds = HMDataset(test_df, SEQ_LEN, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


def inference(model, loader, k=12):
    model.eval()
    
    tbar = tqdm(loader, file=sys.stdout)
    
    preds = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            logits = model(inputs)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                preds.append(" ".join(list(le_article.inverse_transform(indices[i]))))
        
    
    return preds


test_df["prediction"] = inference(model, test_loader)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 5360/5360 [14:21<00:00,  6.22it/s]


In [22]:
test_df.to_csv("submission.csv", index=False, columns=["customer_id", "prediction"])

In [23]:
# fin!